In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np

# -------------------------------
# 1. Data Loading & Preprocessing
# -------------------------------
nltk.download('vader_lexicon')


df = pd.read_csv('data/otsego_good_maybe.csv', parse_dates=['timestamp'])
# Convert 'timestamp' to datetime and drop null timestamps
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df[df['timestamp'].notnull()]
# Convert empty or na contents to string
df['content'] = df['content'].fillna("").astype(str)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/uz/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# -------------------------------
# 2. Sentiment Analysis: VADER
# -------------------------------
vader = SentimentIntensityAnalyzer()
df['vader_compound'] = df['content'].apply(lambda x: vader.polarity_scores(x)['compound'])

# -------------------------------
# 3. Sentiment Analysis: BERT
# -------------------------------
# Initialize a BERT sentiment-analysis pipeline.
bert_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def get_bert_compound(text):
    # Get the sentiment result (returns a list of dicts)
    result = bert_pipeline(text)
    # Extract the star rating from the label (e.g., "4 stars")
    label = result[0]['label']
    rating = int(label.split()[0])
    # Map rating (1-5) to a compound score between -1 and 1:
    # 1 -> -1.0, 2 -> -0.5, 3 -> 0.0, 4 -> 0.5, 5 -> 1.0
    compound = (rating - 3) / 2.0
    return compound

# Apply BERT sentiment analysis (this may take a bit, depending on the dataset size)
df['bert_compound'] = df['content'].apply(get_bert_compound)

NameError: name 'init_empty_weights' is not defined